In [48]:
import pandas as pd
import numpy as np

pd.options.display.float_format = '{:.4f}'.format
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [49]:
# stata intermediate data outputs
full_data_stata = pd.read_stata("../../../stata_files/full_data.dta")
before_mcp_stata = pd.read_stata("../../../stata_files/before_mcp_matrix.dta")
rca_stata = pd.read_stata("../../../stata_files/mcp_rca_matrix.dta")
mcp_stata = pd.read_stata("../../../stata_files/mcp_matrix.dta")
ecomp_stata = pd.read_stata("../../../stata_files/ecomplexity.dta")
reliable_stata = pd.read_stata("../../../stata_files/complexity_reliable_countries.dta")
all_countries_stata = pd.read_stata("../../../stata_files/complexity_all_countries.dta")
allcp_stata = pd.read_stata("../../../stata_files/complexity_all_countries_all_products.dta")
comp_stata = pd.read_stata("../../../stata_files/Complexity_H0_2015.dta")

In [50]:
# python intermediate data outputs
full_data_py = pd.read_parquet("../data/intermediate/H0_2015_complexity_all_countries.parquet")
before_mcp_py = pd.read_parquet("../data/intermediate/H0_2015_before_mcp.parquet")
rca_py = pd.read_parquet("../data/intermediate/H0_2015_mcp_rca.parquet")
mcp_py = pd.read_parquet("../data/intermediate/H0_2015_mcp.parquet")
ecomp_py = pd.read_parquet("../data/intermediate/H0_2015_complexitytest.parquet")
reliable_py = pd.read_parquet("../data/intermediate/H0_2015_reliable_countries.parquet")
all_countries_py = pd.read_parquet("../data/intermediate/H0_2015_all_countries.parquet")
allcp_py = pd.read_parquet("../data/intermediate/H0_2015_all_countries_all_products.parquet")
comp_py = pd.read_parquet("../data/processed/H0_2015_complexity.parquet")

In [51]:
def stata_python_data_review(df_stata, df_py, merge_on, value_checks):
    print(f"stata df shape: {df_stata.shape} py df shape {df_py.shape}")
    merged = df_py.merge(df_stata, on=merge_on, how='outer', suffixes=('_py', '_stata'))
    errors = pd.DataFrame()
    for col in value_checks: 
        print(f"reviewing {col}")
        merged[f'{col}_diff'] = merged[f'{col}_stata'] - merged[f'{col}_py']
        print(f"floating point precision is met: {np.isclose(merged[f'{col}_stata'], merged[f'{col}_py']).all() == True}")
        not_close = merged[~np.isclose(merged[f'{col}_stata'], merged[f'{col}_py'])]
        errors = pd.concat([errors, not_close])
    return merged, errors
    

In [5]:
# full data 
full_data_stata = full_data_stata.drop(columns='year')
full_data_merged, not_close = stata_python_data_review(full_data_stata, full_data_py, ['exporter', 'commoditycode'], ['export_value', 'import_value'])

stata df shape: (289224, 7) py df shape (289224, 7)
reviewing export_value
floating point precision is met: True
reviewing import_value
floating point precision is met: True


In [6]:
# before mcp
before_mcp_stata.columns, before_mcp_py.columns
before_mcp_stata.shape, before_mcp_py.shape
before_mcp_merged, not_close = stata_python_data_review(before_mcp_stata, before_mcp_py, ['exporter', 'commoditycode'], ['export_value'])

stata df shape: (163989, 5) py df shape (163989, 5)
reviewing export_value
floating point precision is met: True


In [7]:
# rca matrix
rename_dict = {"HH": "HH_index"}
rca_stata = rca_stata.rename(columns = rename_dict)
rca_merged, not_close_rca = stata_python_data_review(rca_stata, rca_py, ['exporter', 'commoditycode'], ['export_value', 'rca', 'mcp', 'HH_index'])

stata df shape: (163989, 11) py df shape (163989, 8)
reviewing export_value
floating point precision is met: True
reviewing rca
floating point precision is met: True
reviewing mcp
floating point precision is met: True
reviewing HH_index
floating point precision is met: True


In [8]:
# mcp
rename_dict = {
    'HH': 'HH_index',
    'cumshare': 'cumul_share',
    'ene': 'eff_exporters',
    'c1': 'flag_for_small_share',
    'c2': 'flag_for_few_exporters',
    'c3': 'flag_for_low_ubiquity',
    'call': 'exclude_flag'
}

mcp_stata = mcp_stata.rename(columns = rename_dict)
mcp_cols = ['commoditycode', 'export_value', 'HH_index', 'mcp', 'share',
        'cumul_share', 'eff_exporters', 'flag_for_small_share',
        'flag_for_few_exporters', 'flag_for_low_ubiquity', 'exclude_flag']
mcp_merged = stata_python_data_review(mcp_stata, mcp_py, ['commoditycode'], ['export_value', 'HH_index', 'mcp', 'share', 'exclude_flag'])

stata df shape: (1233, 11) py df shape (1233, 11)
reviewing export_value
floating point precision is met: True
reviewing HH_index
floating point precision is met: True
reviewing mcp
floating point precision is met: True
reviewing share
floating point precision is met: True
reviewing exclude_flag
floating point precision is met: True


In [9]:
# ecomp
ecomp_py.shape, ecomp_stata.shape
ecomp_py.columns, ecomp_stata.columns
ecomp_stata = ecomp_stata.rename(columns={'M': 'mcp'})
ecomp_stata = ecomp_stata.astype({
    'exporter': 'object',
    'commoditycode': 'object',
    'export_value': 'float32',
    'diversity': 'int64',
    'ubiquity': 'int64',
    'mcp': 'int64',
    'eci': 'float64',
    'pci': 'float64',
    'density': 'float64',
    'coi': 'float64',
    'cog': 'float64',
    'rca': 'float32'
})
ecomp_merged, not_close_ecomp = stata_python_data_review(ecomp_stata, ecomp_py, ['exporter', 'commoditycode'], ['export_value', 'diversity', 'ubiquity', 'mcp', 'eci', 'pci', 'density', 'coi', 'cog', 'rca'])

stata df shape: (150822, 14) py df shape (150822, 14)
reviewing export_value
floating point precision is met: True
reviewing diversity
floating point precision is met: True
reviewing ubiquity
floating point precision is met: True
reviewing mcp
floating point precision is met: True
reviewing eci
floating point precision is met: True
reviewing pci
floating point precision is met: False
reviewing density
floating point precision is met: False
reviewing coi
floating point precision is met: False
reviewing cog
floating point precision is met: False
reviewing rca
floating point precision is met: True


In [10]:
not_close_ecomp.columns

Index(['exporter', 'commoditycode', 'export_value_py', 'year', 'diversity_py',
       'ubiquity_py', 'mcp_py', 'eci_py', 'pci_normalized', 'density_py',
       'coi_py', 'cog_py', 'rca_py', 'pci_py', 'export_value_stata',
       'population', 'gdp_pc', 'rca_stata', 'mcp_stata', 'density_stata',
       'eci_stata', 'pci_stata', 'diversity_stata', 'ubiquity_stata',
       'coi_stata', 'cog_stata', 'export_value_diff', 'diversity_diff',
       'ubiquity_diff', 'mcp_diff', 'eci_diff', 'pci_diff', 'density_diff',
       'coi_diff', 'cog_diff', 'rca_diff'],
      dtype='object')

In [11]:
cols = ['year', 'exporter', 'commoditycode', 
          'export_value_py', 'export_value_stata', 'export_value_diff',
          'diversity_py', 'diversity_stata', 'diversity_diff',
          'ubiquity_py', 'ubiquity_stata', 'ubiquity_diff',
          'mcp_py', 'mcp_stata', 'mcp_diff',
          'eci_py', 'eci_stata', 'eci_diff',
          'pci_normalized', 'pci_py', 'pci_stata', 'pci_diff',
          'density_py', 'density_stata', 'density_diff',
          'coi_py', 'coi_stata', 'coi_diff',
          'cog_py', 'cog_stata', 'cog_diff',
          'rca_py', 'rca_stata', 'rca_diff',
          'population', 'gdp_pc']
not_close_ecomp = not_close_ecomp[cols]
ecomp_merged = ecomp_merged[cols]
ecomp_merged[ecomp_merged.pci_diff>0.1]

,year,exporter,commoditycode,export_value_py,export_value_stata,export_value_diff,diversity_py,diversity_stata,diversity_diff,ubiquity_py,ubiquity_stata,ubiquity_diff,mcp_py,mcp_stata,mcp_diff,eci_py,eci_stata,eci_diff,pci_normalized,pci_py,pci_stata,pci_diff,density_py,density_stata,density_diff,coi_py,coi_stata,coi_diff,cog_py,cog_stata,cog_diff,rca_py,rca_stata,rca_diff,population,gdp_pc
0,2015,AGO,0101,0.00000,0.00000,0.00000,8,8,0,15,15,0,0,0,0,-1.11344,-1.11343,0.00000,0.74048,0.00330,0.74048,0.73718,0.00581,0.00585,0.00004,-1.19944,-1.19787,0.00157,0.53474,0.53540,0.00066,0.00000,0.00000,0.00000,28127720.00000,3100.83057
2,2015,AGO,0103,0.00000,0.00000,0.00000,8,8,0,17,17,0,0,0,0,-1.11344,-1.11343,0.00000,1.88472,0.02150,1.88472,1.86322,0.00582,0.00585,0.00003,-1.19944,-1.19787,0.00157,0.57455,0.57052,-0.00403,0.00000,0.00000,0.00000,28127720.00000,3100.83057
4,2015,AGO,0105,9279.83789,9279.83789,0.00000,8,8,0,33,33,0,0,0,0,-1.11344,-1.11343,0.00000,0.91110,0.00602,0.91110,0.90508,0.00591,0.00594,0.00002,-1.19944,-1.19787,0.00157,0.34266,0.34172,-0.00094,0.00149,0.00149,0.00000,28127720.00000,3100.83057
6,2015,AGO,0201,5859.69287,5859.69287,0.00000,8,8,0,28,28,0,0,0,0,-1.11344,-1.11343,0.00000,0.57829,0.00072,0.57828,0.57756,0.00732,0.00736,0.00003,-1.19944,-1.19787,0.00157,0.24778,0.24735,-0.00043,0.00013,0.00013,0.00000,28127720.00000,3100.83057
8,2015,AGO,0203,111847.07812,111847.08594,0.00781,8,8,0,17,17,0,0,0,0,-1.11344,-1.11343,0.00000,2.24904,0.02729,2.24903,2.22174,0.00543,0.00545,0.00002,-1.19944,-1.19787,0.00157,0.88086,0.87724,-0.00362,0.00209,0.00209,-0.00000,28127720.00000,3100.83057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150815,2015,ZWE,9618,0.00000,0.00000,0.00000,85,85,0,12,12,0,0,0,0,-0.79126,-0.79126,0.00000,1.91101,0.02192,1.91100,1.88908,0.05087,0.05113,0.00025,-0.91809,-0.91857,-0.00048,0.74727,0.74374,-0.00353,0.00000,0.00000,0.00000,14154937.00000,1410.32922
150816,2015,ZWE,9701,139873.84375,139873.84375,0.00000,85,85,0,5,5,0,0,0,0,-0.79126,-0.79126,0.00000,4.97747,0.07068,4.97745,4.90677,0.03933,0.03969,0.00036,-0.91809,-0.91857,-0.00048,0.93843,0.90353,-0.03490,0.04541,0.04541,0.00000,14154937.00000,1410.32922
150817,2015,ZWE,9702,1854.63196,1854.63196,0.00000,85,85,0,8,8,0,0,0,0,-0.79126,-0.79126,0.00000,3.22619,0.04283,3.22618,3.18334,0.05890,0.05939,0.00049,-0.91809,-0.91857,-0.00048,0.80128,0.78334,-0.01794,0.02096,0.02096,0.00000,14154937.00000,1410.32922
150818,2015,ZWE,9703,1329172.25000,1329172.25000,0.00000,85,85,0,11,11,0,1,1,0,-0.79126,-0.79126,0.00000,0.85515,0.00513,0.85514,0.85002,0.07952,0.07313,-0.00638,-0.91809,-0.91857,-0.00048,0.00000,0.00000,0.00000,2.08140,2.08140,-0.00000,14154937.00000,1410.32922


In [12]:
# reliable country set
reliable_stata.columns, reliable_py.columns
rename_stata = {"rca1": "rca_reliable", "M": "mcp", "eci1": "eci_reliable", "pci1": "pci_reliable", "density1": "density"}
reliable_stata = reliable_stata.rename(columns=rename_stata)
reliable_merged, not_close_reliable = stata_python_data_review(reliable_stata, reliable_py, ['exporter', 'commoditycode'], ['eci_reliable', 'pci_reliable', 'rca_reliable'])

stata df shape: (150822, 14) py df shape (150822, 20)
reviewing eci_reliable
floating point precision is met: True
reviewing pci_reliable
floating point precision is met: True
reviewing rca_reliable
floating point precision is met: True


In [13]:
all_countries_stata.head()

,exporter,commoditycode,export_value,population,gdp_pc,rca1,M,density1,eci1,pci1,diversity,ubiquity,coi,cog
0,AGO,0101,0.00000,28127720.00000,3100.83057,0.00000,0.00000,0.00585,-1.11343,-0.13481,8.00000,15.00000,-1.19787,0.53540
1,AGO,0102,1114.02429,28127720.00000,3100.83057,0.00006,0.00000,0.00884,-1.11343,-0.64367,8.00000,41.00000,-1.19787,-0.05637
2,AGO,0103,0.00000,28127720.00000,3100.83057,0.00000,0.00000,0.00585,-1.11343,0.49645,8.00000,17.00000,-1.19787,0.57052
3,AGO,0104,0.00000,28127720.00000,3100.83057,0.00000,0.00000,0.01076,-1.11343,-1.41822,8.00000,21.00000,-1.19787,-0.18328
4,AGO,0105,9279.83789,28127720.00000,3100.83057,0.00149,0.00000,0.00594,-1.11343,-0.04068,8.00000,33.00000,-1.19787,0.34172


In [14]:
all_countries_py.head()

,exporter,commoditycode,export_value,import_value,rca,mcp,pci_reliable,pci,eci,prody,expy
0,ABW,0101,14128.37402,25784.36328,0.21492,0,-0.13481,0.00330,0.80134,33575.17847,16663.02084
1,ABW,0102,0.00000,0.00000,0.00000,0,-0.64367,-0.01136,0.80134,10189.51052,16663.02084
2,ABW,0103,0.00000,0.00000,0.00000,0,0.49645,0.02150,0.80134,34897.09809,16663.02084
3,ABW,0104,0.00000,5445.36768,0.00000,0,-1.41822,-0.03369,0.80134,2892.97447,16663.02084
4,ABW,0105,123889.79688,285076.37500,1.69645,1,-0.04068,0.00602,0.80134,17629.68156,16663.02084


In [15]:
# all_countries
all_countries_stata = all_countries_stata.rename(columns={"eci2": "eci", "rca2": "rca", "pci1":"pci_reliable"})
comp_merged, errors = stata_python_data_review(all_countries_stata, all_countries_py, ['exporter', 'commoditycode'], ['rca', 'eci', 'prody', 'pci_reliable'])

stata df shape: (150822, 14) py df shape (265356, 11)
reviewing rca


KeyError: 'rca_stata'

In [ ]:
errors.sort_values(by="pci_reliable_stata")

In [52]:
allcp_stata.head()

,exporter,commoditycode,export_value,rca3,rca2,rca1,pci3,pci1,eci2,eci1,import_value,gdp_pc,M,density3,prody3,opportunity_value,opportunity_gain,expy,prody,density1,coi,cog,tagc,tagp
0,ABW,0101,14128.3740,0.2167,0.2149,NaN,-0.2584,NaN,0.8013,NaN,25784.3633,28421.3870,0,0.0859,35436.4688,-0.0034,0.0013,16663.0215,33575.1797,NaN,NaN,NaN,1,1
1,ABW,0102,0.0000,0.0000,0.0000,NaN,-0.8390,NaN,0.8013,NaN,0.0000,28421.3870,0,0.0902,5726.1353,-0.0034,-0.0008,16663.0215,10189.5098,NaN,NaN,NaN,0,1
2,ABW,0103,0.0000,0.0000,0.0000,NaN,0.6966,NaN,0.8013,NaN,0.0000,28421.3870,0,0.0738,31966.9980,-0.0034,0.0020,16663.0215,34897.0977,NaN,NaN,NaN,0,1
3,ABW,0104,0.0000,0.0000,0.0000,NaN,-1.9519,NaN,0.8013,NaN,5445.3677,28421.3870,0,0.0787,1271.8053,-0.0034,-0.0012,16663.0215,2892.9746,NaN,NaN,NaN,0,1
4,ABW,0105,123889.7969,1.7109,1.6965,NaN,-0.0734,NaN,0.8013,NaN,285076.3750,28421.3870,1,0.0942,17453.0273,-0.0034,0.0000,16663.0215,17629.6816,NaN,NaN,NaN,0,1


In [53]:
allcp_py.head()

,exporter,commoditycode,export_value,gdp_pc,import_value,rca,mcp,eci,pci,prody,density,opportunity_value,opportunity_gain
0,ABW,0101,14128.3740,28421.3870,25784.3633,0.2167,0,0.0055,-0.0004,35436.4698,0.0863,0.1371,0.0011
1,ABW,0102,0.0000,28421.3870,0.0000,0.0000,0,0.0055,-0.0047,5726.1354,0.0776,0.1371,0.0003
2,ABW,0103,0.0000,28421.3870,0.0000,0.0000,0,0.0055,0.0067,31966.9972,0.0793,0.1371,0.0016
3,ABW,0104,0.0000,28421.3870,5445.3677,0.0000,0,0.0055,-0.0129,1271.8052,0.0776,0.1371,-0.0010
4,ABW,0105,123889.7969,28421.3870,285076.3750,1.7109,1,0.0055,0.0010,17453.0280,0.0851,0.1371,0.0000


In [44]:
# all_countries, all products
allcp_py.columns, allcp_stata.columns
# allcp_py.dtypes, allcp_stata.dtypes
allcp_py = allcp_py.rename(columns={"prody":"prody_allcp"})
allcp_stata = allcp_stata.rename(columns={"pci3": "pci", "rca3": "rca", "prody3" : "prody_allcp", "M":"mcp", "eci2":"eci_all_countries"})
allcp_merged, allcp_errors = stata_python_data_review(allcp_stata, allcp_py, ['exporter', 'commoditycode'], ['pci', 'rca','prody_allcp'])

stata df shape: (289224, 24) py df shape (289224, 10)
reviewing pci
floating point precision is met: False
reviewing rca
floating point precision is met: False
reviewing prody_allcp
floating point precision is met: True


In [39]:
allcp_errors.head()

,exporter,commoditycode,export_value_py,gdp_pc_py,import_value_py,rca_py,mcp_py,eci_all_countries_py,pci_py,prody_allcp_py,export_value_stata,rca_stata,rca2,rca1,pci_stata,pci1,eci_all_countries_stata,eci1,import_value_stata,gdp_pc_stata,mcp_stata,density3,prody_allcp_stata,opportunity_value,opportunity_gain,expy,prody,density1,coi,cog,tagc,tagp,pci_diff,rca_diff,prody_allcp_diff
0,ABW,0101,14128.37402,28421.38700,25784.36328,0.21674,0,0.80134,0.42815,35436.46976,14128.37402,0.21674,0.21492,NaN,-0.25841,NaN,0.80134,NaN,25784.36328,28421.38700,0,0.08592,35436.46875,-0.00337,0.00127,16663.02148,33575.17969,NaN,NaN,NaN,1,1,-0.68656,NaN,NaN
1,ABW,0102,0.00000,28421.38700,0.00000,0.00000,0,0.80134,0.15396,5726.13541,0.00000,0.00000,0.00000,NaN,-0.83901,NaN,0.80134,NaN,0.00000,28421.38700,0,0.09023,5726.13525,-0.00337,-0.00081,16663.02148,10189.50977,NaN,NaN,NaN,0,1,-0.99297,NaN,NaN
2,ABW,0103,0.00000,28421.38700,0.00000,0.00000,0,0.80134,0.87917,31966.99723,0.00000,0.00000,0.00000,NaN,0.69662,NaN,0.80134,NaN,0.00000,28421.38700,0,0.07383,31966.99805,-0.00337,0.00201,16663.02148,34897.09766,NaN,NaN,NaN,0,1,-0.18255,NaN,NaN
3,ABW,0104,0.00000,28421.38700,5445.36768,0.00000,0,0.80134,-0.37161,1271.80525,0.00000,0.00000,0.00000,NaN,-1.95191,NaN,0.80134,NaN,5445.36768,28421.38700,0,0.07873,1271.80530,-0.00337,-0.00117,16663.02148,2892.97461,NaN,NaN,NaN,0,1,-1.58030,NaN,NaN
4,ABW,0105,123889.79688,28421.38700,285076.37500,1.71085,1,0.80134,0.51551,17453.02799,123889.79688,1.71085,1.69645,NaN,-0.07342,NaN,0.80134,NaN,285076.37500,28421.38700,1,0.09417,17453.02734,-0.00337,0.00000,16663.02148,17629.68164,NaN,NaN,NaN,0,1,-0.58893,NaN,NaN


In [54]:
# complexity set
comp_merged, not_close_comp = stata_python_data_review(comp_stata, comp_py, ['year', 'exporter', 'commoditycode'], ['rca', 'mcp', 'eci', 'pci', 'oppval', 'oppgain', 'distance', 'prody', 'expy'])

stata df shape: (289224, 16) py df shape (288522, 16)
reviewing rca
floating point precision is met: False
reviewing mcp
floating point precision is met: False
reviewing eci
floating point precision is met: False
reviewing pci
floating point precision is met: False
reviewing oppval
floating point precision is met: False
reviewing oppgain
floating point precision is met: False
reviewing distance
floating point precision is met: False
reviewing prody
floating point precision is met: False
reviewing expy
floating point precision is met: False


In [55]:
not_close_comp.head()

,year,exporter,commoditycode,inatlas_py,export_value_py,import_value_py,rca_py,mcp_py,eci_py,pci_py,oppval_py,oppgain_py,distance_py,prody_py,expy_py,gdp_pc_py,inatlas_stata,export_value_stata,import_value_stata,rca_stata,mcp_stata,eci_stata,pci_stata,oppval_stata,oppgain_stata,distance_stata,prody_stata,expy_stata,gdp_pc_stata,rca_diff,mcp_diff,eci_diff,pci_diff,oppval_diff,oppgain_diff,distance_diff,prody_diff,expy_diff
288522,2015,ABW,7108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1080323.8750,42308.0820,0.1383,0,0.8013,-2.1466,-0.0034,-0.0020,0.9005,7380.8862,16663.0215,28421.3870,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
288523,2015,ABW,9999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,19327292.0000,81078064.0000,1.3480,1,0.8013,-0.2761,-0.0034,0.0000,0.8959,16716.9648,16663.0215,28421.3870,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
288524,2015,ABW,XXXX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0000,0.0000,0.0000,1,0.8013,-1.1650,-0.0034,0.0000,0.9036,0.0000,16663.0215,28421.3870,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
288525,2015,AFG,7108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1002309.8750,71353984.0000,0.0619,0,-1.1785,-2.1466,-0.8867,-0.0014,0.8934,7380.8862,7718.3755,592.4761,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
288526,2015,AFG,9999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,36940396.0000,1167553408.0000,1.2424,1,-1.1785,-0.2761,-0.8867,0.0000,0.9254,16716.9648,7718.3755,592.4761,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
not_close_comp.head()

,year,exporter,commoditycode,inatlas_py,export_value_py,import_value_py,rca_py,mcp_py,eci_py,pci_py,oppval_py,oppgain_py,distance_py,prody_py,expy_py,gdp_pc_py,inatlas_stata,export_value_stata,import_value_stata,rca_stata,mcp_stata,eci_stata,pci_stata,oppval_stata,oppgain_stata,distance_stata,prody_stata,expy_stata,gdp_pc_stata,rca_diff,mcp_diff,eci_diff,pci_diff,oppval_diff,oppgain_diff,distance_diff,prody_diff,expy_diff
288522,2015,ABW,7108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1080323.87500,42308.08203,0.13833,0,0.80134,-2.14658,-0.00337,-0.00198,0.90047,7380.88623,16663.02148,28421.38700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
288523,2015,ABW,9999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,19327292.00000,81078064.00000,1.34803,1,0.80134,-0.27609,-0.00337,0.00000,0.89588,16716.96484,16663.02148,28421.38700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
288524,2015,ABW,XXXX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.00000,0.00000,0.00000,1,0.80134,-1.16502,-0.00337,0.00000,0.90358,0.00000,16663.02148,28421.38700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
288525,2015,AFG,7108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1002309.87500,71353984.00000,0.06189,0,-1.17852,-2.14658,-0.88667,-0.00143,0.89339,7380.88623,7718.37549,592.47614,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
288526,2015,AFG,9999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,36940396.00000,1167553408.00000,1.24236,1,-1.17852,-0.27609,-0.88667,0.00000,0.92544,16716.96484,7718.37549,592.47614,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
reliabl